# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [1]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
import sklearn as sk

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from loguru import logger

In [2]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [3]:
fifa = pd.read_csv("fifa.csv")

In [4]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [5]:
# Sua análise começa aqui.


In [6]:
fifa.shape

(18207, 37)

In [7]:
fifa.head()

,Age,Overall,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,31,94,94,84.0,95.0,70.0,90.0,86.0,97.0,93.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,33,94,94,84.0,94.0,89.0,81.0,87.0,88.0,81.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,26,92,93,79.0,87.0,62.0,84.0,84.0,96.0,88.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,27,91,93,17.0,13.0,21.0,50.0,13.0,18.0,21.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,27,91,92,93.0,82.0,55.0,92.0,82.0,86.0,85.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [8]:
#Criando um dataframe auxliar para analisar a consistencia das variaveis
df = pd.DataFrame({'colunas' : fifa.columns,
                    'tipo': fifa.dtypes,
                    'missing' : fifa.isna().sum(),
                    'size' : fifa.shape[0],
                    'unicos': fifa.nunique()})
df['percentual'] = round(df['missing'] / df['size'],2)

In [9]:
df

,colunas,tipo,missing,size,unicos,percentual
Age,Age,int64,0,18207,29,0.0
Overall,Overall,int64,0,18207,48,0.0
Potential,Potential,int64,0,18207,47,0.0
Crossing,Crossing,float64,48,18207,89,0.0
Finishing,Finishing,float64,48,18207,93,0.0
HeadingAccuracy,HeadingAccuracy,float64,48,18207,91,0.0
ShortPassing,ShortPassing,float64,48,18207,85,0.0
Volleys,Volleys,float64,48,18207,87,0.0
Dribbling,Dribbling,float64,48,18207,94,0.0
Curve,Curve,float64,48,18207,89,0.0


In [10]:
# Dropar valores nulos pois são poucos!
fifa.dropna(inplace=True)

In [11]:
fifa.shape

(18159, 37)

## Exercicio 01

In [12]:
#instanciando PCA na variável pca.
pca = PCA()

In [13]:
# Treinando meu dataframe
pca.fit(fifa)

PCA()

In [14]:
evr = pca.explained_variance_ratio_

In [15]:
evr

array([5.65280555e-01, 1.81025219e-01, 5.95283391e-02, 4.34269284e-02,
       2.63532326e-02, 1.20707695e-02, 1.03550495e-02, 8.52951131e-03,
       7.87862979e-03, 7.20416677e-03, 7.18180297e-03, 6.32893542e-03,
       5.19242286e-03, 4.90798293e-03, 4.74209042e-03, 4.62772008e-03,
       4.25173246e-03, 3.86288432e-03, 3.76588448e-03, 3.75092615e-03,
       3.47815213e-03, 3.26730709e-03, 3.03137944e-03, 2.97415845e-03,
       2.56797543e-03, 2.30353342e-03, 2.14815132e-03, 1.56135904e-03,
       1.55351696e-03, 1.08664227e-03, 1.06824189e-03, 1.05891525e-03,
       9.58143746e-04, 8.89209394e-04, 8.59732430e-04, 7.34611045e-04,
       1.94187517e-04])

In [16]:
evr[0]

0.5652805550008518

## Exercicio 02

In [17]:
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
component_number = np.argmax(cumulative_variance_ratio >= 0.95) + 1 # Contagem começa em zero.

component_number

15

In [18]:
evr_acumulado = np.cumsum(evr)

In [19]:
num_features = np.argmax(evr_acumulado >= 0.95) + 1 # Contagem começa em zero.

In [20]:
num_features

15

## Exercicio 03

In [21]:
# definindo objeto PCA com PC1 e PC2
pca_componentes = PCA(n_components=2)

In [22]:
#passo os dados (dataframe) para o objeto de pca para reduzir
pca_componentes.fit(fifa)

PCA(n_components=2)

In [23]:
#PC = pca_componentes.components_.dot(x)

In [24]:
#tuple(PC.round(3))

## Exercicio 04

In [25]:
#Separando a variavel target e as labels
target = fifa['Overall']
label = fifa.drop(['Overall'],axis=1)

In [39]:
# Criando um modelo de regressao linear
lr = LinearRegression()

In [40]:
# Criando um RFE para selecionar as n_features melhores para o modelo
rfe = RFE(lr, n_features_to_select=5, step=1)

In [41]:
# Fittando o modelo com os dados de X,y
rfe = selector.fit(label, target)

In [42]:
rfe_suport = rfe.get_support()

In [43]:
rfe_suport

array([ True,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True])

In [46]:
# Com esse print você consegue a lista das variáveis sem ser em true ou false.
print(fifa.loc[:, rfe_suport].columns.tolist())  

['Age', 'Overall', 'LongPassing', 'Agility', 'GKPositioning']


In [31]:
list(label.columns[selector.support_])

['Age', 'Potential', 'BallControl', 'Reactions', 'GKReflexes']

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [32]:
def q1():
    # Retorne aqui o resultado da questão 1.

    # instanciando PCA na variável pca.
    pca = PCA() 
    # passo os dados (dataframe) para o objeto de pca para reduzir
    pca.fit(fifa)
    # extraindo variancia explicada
    evr = pca.explained_variance_ratio_
    return float (round(evr[0],3))

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [33]:
def q2():
    # Retorne aqui o resultado da questão 2.
    
    # Descobrindo o EVR acumulado
    evr_acumulado = np.cumsum(evr)
    # Qual o máximo acumulado antes ou igual a 0.95 (95% da variancia total)
    num_features = np.argmax(evr_acumulado >= 0.95) + 1 # Contagem começa em zero, portanto -> +1.
    return int (num_features)

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [34]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [35]:
def q3():
    # Retorne aqui o resultado da questão 3.

    # definindo objeto PCA com PC1 e PC2
    pca_components = PCA(n_components=2)
    # passo os dados (dataframe) para o objeto de pca para reduzir
    pca_components.fit(fifa)
    # componentes principais
    PCs = pca_components.components_.dot(x)
    return tuple(PCs.round(3))

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [36]:
def q4():
    # Retorne aqui o resultado da questão 4.

    #Separando a variavel target e as labels
    target = fifa['Overall']
    label = fifa.drop(['Overall'],axis=1)
    # Criando um modelo de regressao linear
    lr = LinearRegression()
    # Criando um RFE para selecionar as n_features melhores para o modelo
    selector = RFE(lr, n_features_to_select=5, step=1)
    # Fittando o modelo com os dados de X,y
    selector = selector.fit(label, target)
    return list(label.columns[selector.support_])